# Simple visual servoing example 

This notebook demonstrates how to setup a simple visual servoing demo where Reachy's arm follow the position of a yellow ball. As to keep the servoing simple, we only use relative positions and movement.

In [ ]:
%matplotlib inline

import cv2
import matplotlib.pyplot as plt

First load Leachy as usual.

In [ ]:
from reachy import Leachy

leachy = Leachy(brunel_hand='/dev/tty.usbmodem14141')

Move it to the base position

In [ ]:
for m in leachy.motors:
    m.compliant = False

leachy.l_elbow_pitch.goto_position(-90, 2)

We now define a very simple ball tracking algorithm using OpenCV. It's filter on a specific RGB range and find the center of the detected area.

In [ ]:
cap = cv2.VideoCapture(0)

def track_ball(img):
    yellow_lower = (0, 125, 150)
    yellow_upper = (100, 255, 255)
    
    mask = cv2.inRange(img, yellow_lower, yellow_upper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    M = cv2.moments(mask)
    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    
    return center

We will use two motors to control the end effector on a plan.

* l_arm_yaw will be used to move along the x axis
* l_elbow_pitch will be used to move along the y axis

We then find the x, y position of the ball, normalized in [-1, 1], and apply it to the motor position.

In [ ]:
X_range, X_offset = 30, 0
Y_range, Y_offset = 30, -90

leachy.l_arm_yaw.moving_speed = 50
leachy.l_elbow_pitch.moving_speed = 50

import time

while True:
    _, img = cap.read()

    x, y = track_ball(img)
    x /= img.shape[1]
    y /= img.shape[0]

    x = x * 2 - 1
    leachy.l_arm_yaw.goal_position = -x * X_range + X_offset
    leachy.l_elbow_pitch.goal_position = y * Y_range + Y_offset 

    time.sleep(1 / 50)

Once we are done, we clean everything.

In [ ]:
cap.release()
leachy.close()